In [45]:
import numpy as np
import math
import pandas as pd
from scipy.stats import qmc

desiredTotalCFDCases = 200
inputParamNames = [
    'deformationMode',
    'mach',
    'reynolds',
    'freestreamTemp',
    'lengthFromFlare',
    'deformationLength'
]

l_bounds = [3, 1e6, 80, 0, 19]
u_bounds = [8, 5e7, 150, 2, 76]

deformationModes = [-2,-1,0,1,2] #user defined deformation modes 
numCasesPerMode = desiredTotalCFDCases/len(deformationModes) # divide CFD case budget by the number of def modes 
seedBaseNumbers = np.arange(0,len(deformationModes)) # create numpy array for LDS seed. seed ensures repeatability 
seedList = [base for base in seedBaseNumbers] #turns np array in list
deformationModesZeroIndex = np.where(np.asarray(deformationModes)==0)[0][0] # finds the index where deformation mode == 0. 

m = math.log2(numCasesPerMode)
m = math.ceil(m)

printDict = {
    True: 'HAVE',
    False: 'HAVE NOT'
}

def is_power_of_two(number : int) -> bool:
    while number != 1:
        if number % 2:
            return False
        number /= 2
    return True

powerBool = is_power_of_two(number=numCasesPerMode) # creates boolean if the number of cases creates a power of 2 for the low discrepancy sequence  

print(f'You {printDict[powerBool]} selected a power of 2 for number of cases.') 
if not powerBool:
    print(f'Based on your desired number of CFD cases, you\'ll need to run {(2**m)*len(deformationModes)} cases to safely use LDS.The next lowest increment is {(2**(m-1))*len(deformationModes)} CFD cases\n')

for i, seed in enumerate(seedList):
    if seed == seedList[deformationModesZeroIndex]: 
        l_bounds_mod = l_bounds[:3] #removing the 'length from flare' and 'deformation length'
        u_bounds_mod = u_bounds[:3] #removing the 'length from flare' and 'deformation length'
        sampler = qmc.Sobol(d=len(l_bounds_mod), scramble=False,seed = seed)
        sample = sampler.random_base2(m=m)
        print(f'discrepancy of sample: {qmc.discrepancy(sample)}. seed: {seed}')
        sample = qmc.scale(sample, l_bounds_mod, u_bounds_mod)
        tiledModes = np.tile(deformationModes[i], 2**m).reshape(-1,1)
        stackedSample = np.hstack((tiledModes,sample))
        zerosArray = np.zeros(shape=(2**m, 2))
        stackedSample = np.hstack((stackedSample,zerosArray))
    else:
        sampler = qmc.Sobol(d=len(l_bounds), scramble=False,seed = seed)
        sample = sampler.random_base2(m=m)
        print(f'discrepancy of sample: {qmc.discrepancy(sample)}. seed: {seed}')
        sample = qmc.scale(sample, l_bounds, u_bounds)
        tiledModes = np.tile(deformationModes[i], 2**m).reshape(-1,1)
        stackedSample = np.hstack((tiledModes,sample))
    if i ==0:
        sampleArray = stackedSample
    else: 
        sampleArray = np.concatenate((sampleArray, stackedSample))


new_data=np.array([[2, 4, 20000000, 128, 1, 38],[1, 4, 20000000, 128, 1, 38],[-2, 4, 20000000, 128, 1, 38],[-1, 4, 20000000, 128, 1, 38],[0, 4, 20000000, 128, 0, 0],[2, 7, 5000000, 93.5, 1, 38],[1, 7, 5000000, 93.5, 1, 38],[-2, 7, 5000000, 93.5, 1, 38],[-1, 7, 5000000, 93.5, 1, 38],[0, 7, 5000000, 93.5, 0, 0]])
sampleArray = np.concatenate((sampleArray,new_data))

print(f'You have {sampleArray.shape[1]} number of input variables and a total of {sampleArray.shape[0]} number of cases.')
print(f'Shape of your input sample array: {sampleArray.shape}')

df = pd.DataFrame(data = sampleArray, columns = inputParamNames)
df.to_csv('./inputVariables.csv')


You HAVE NOT selected a power of 2 for number of cases.
Based on your desired number of CFD cases, you'll need to run 320 cases to safely use LDS.The next lowest increment is 160 CFD cases

discrepancy of sample: 0.0031211288638202106. seed: 0
discrepancy of sample: 0.0031211288638202106. seed: 1
discrepancy of sample: 0.0005947820123695102. seed: 2
discrepancy of sample: 0.0031211288638202106. seed: 3
discrepancy of sample: 0.0031211288638202106. seed: 4
You have 6 number of input variables and a total of 330 number of cases.
Shape of your input sample array: (330, 6)
